In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


def data_generator(csv_file_path):
    data_all = pd.read_csv(csv_file_path)
    data_all.set_index('Date', inplace=True)
    data_all.index = pd.to_datetime(data_all.index)
    data_all['Buy/Sell'] = np.ones(data_all.shape[0])
    data_all.drop('Adj Close', axis=1, inplace=True)

    data = data_all.iloc[20:, :]

    five_days_move = data.pct_change(periods=5).dropna(axis=0)
    long_date_dat = five_days_move[five_days_move['Close'] >= 0.05]
    short_date_dat = five_days_move[five_days_move['Close'] <= -0.05]

    x = []
    y = []
    buy_n = 0
    short_n = 0

    for i in range(len(long_date_dat.index)):
        five_days = data_all.loc[:long_date_dat.index[i]]
        cost = data_all.loc[five_days.iloc[-6:-5, :].index]['Close'].values
        buy = True
        stop_loss = 0
        for closes in five_days.iloc[-6:-1]['Close']:
            if closes < cost - stop_loss:
                buy = False
        if buy:
            y.append(str(five_days.iloc[-6:-5, :].index)[16:26])
            buy_n += 1
    for i in range(len(short_date_dat.index)):
        five_days = data_all.loc[:short_date_dat.index[i]]
        cost = data_all.loc[five_days.iloc[-6:-5, :].index]['Close'].values
        buy = True
        stop_loss = 0
        for closes in five_days.iloc[-6:-1]['Close']:
            if closes > cost - stop_loss:
                buy = False
        if buy:
            y.append(str(five_days.iloc[-6:-5, :].index)[16:26])
            short_n += 1
    for i in range(len(y)):
        twenty_days = data_all.loc[:y[i]].iloc[-21:-1, :].index
        temp_x = []
        for j in range(len(twenty_days)):
            temp_x.append(str(twenty_days[j])[:10])
        x.append(temp_x)

    for i in range(buy_n):
        y[i] = 1
    for i in range(buy_n, len(y)):
        y[i] = 0

    y = np.array(y)

    x_stats = []
    sc = MinMaxScaler(feature_range=(0, 1))
    for dates in x:
        x_stats.append(sc.fit_transform(data_all.loc[dates, :'Volume'].values))

    x_stats = np.dstack(x_stats)
    x_stats = np.rollaxis(x_stats, -1)
    return x_stats, y



In [3]:
from keras import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Bidirectional
from keras.layers import LSTM
import random

x, y = data_generator('Data/^IXIC.csv')

# Shuffle Data
temp = list(zip(x, y))
random.shuffle(temp)
x, y = zip(*temp)
x, y = np.array(x), np.array(y)
y = np.expand_dims(y, axis=1)

# Split Into Train Test Set
x_train, x_test, y_train, y_test = x[:260], x[260:], y[:260, :], y[260:, :]


def lstm_model(input_dim, output_dim, feature_size):

    model_lstm = Sequential()
    model_lstm.add(Bidirectional(LSTM(units=50), input_shape=(input_dim, feature_size)))
    model_lstm.add(Dense(25))
    model_lstm.add(Dense(units=output_dim))
    model_lstm.compile(optimizer='adam', loss='mse')
    model_lstm.fit(x_train, y_train, epochs=100, batch_size=1, validation_data=(x_test, y_test), verbose=1)

    return model_lstm


model = lstm_model(20, 1, 5)
print(model.summary())
print(model.predict(x_test))

[[[1.         1.         1.         1.         0.06417269]
  [0.93803967 0.95073672 0.92599306 0.89982221 0.04628773]
  [0.75754085 0.75318749 0.70465703 0.68935614 0.28402769]
  ...
  [0.14954974 0.19922009 0.24677614 0.30360131 0.49125663]
  [0.15948645 0.19284561 0.20118784 0.19961153 0.15077478]
  [0.         0.         0.         0.         1.        ]]

 [[1.         1.         1.         1.         0.28402769]
  [0.73135994 0.71819653 0.71814311 0.68742444 0.50387098]
  [0.61510924 0.74463467 0.74147322 0.90820812 0.44132632]
  ...
  [0.1970642  0.14275658 0.09791587 0.08261257 1.        ]
  [0.         0.         0.         0.         0.87357547]
  [0.09114853 0.20549422 0.20492659 0.35315706 0.71842022]]

 [[0.93779196 0.91988448 0.94809264 0.75690189 0.50387098]
  [0.78872861 0.95374713 0.97889306 1.         0.44132632]
  [0.78833501 0.993384   0.98130088 0.88645655 0.50082573]
  ...
  [0.         0.         0.         0.         0.87357547]
  [0.1168759  0.26320226 0.2705441